In [1]:
import pandas as pd 
df=pd.read_csv('/scratch/user/zshuying/ppi_mutation/data/2019_1_sequences_terminated.csv')

# longest protein in current train/val delta set

In [8]:
import os, sys
global top_path  # the path of the top_level directory
global script_path, data_path, logging_path
top_path = '/scratch/user/zshuying/ppi_mutation'
sys.path.append(top_path)
from scripts.myesm.model import*
from scripts.myesm.datasets import *

/scratch/user/zshuying/.conda/envs/yuning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
proData=ProteinDataModule(train_val_ratio=0.8,low=0,medium=512,high=1028,veryhigh=1500,discard=True,num_devices=1,num_nodes=1,delta=True,bs_short=2,bs_medium=1)

Reading file
nan removed
count    13769.000000
mean         0.648123
std          0.477573
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Label, dtype: float64
Split dataset into train, val with the rate of 0.8
Splitting training set by length
Discarded 1809 sequences which are longer than 1500
short dataset cut to length between 0 and 512
sequences count = 4196
----------------------------------------------
medium dataset cut to length between 512 and 1028
sequences count = 3746
----------------------------------------------
long dataset cut to length between 1028 and 1500
sequences count = 1264
----------------------------------------------
Splitting validation set by length
Discarded 454 sequences which are longer than 1500
short dataset cut to length between 0 and 512
sequences count = 1033
----------------------------------------------
medium dataset cut to length between 512 and 1028
sequences count = 884
------

In [15]:
a=proData.dataset.all_sequences.Seq.str.len()<1500

In [17]:
esm_model, alphabet=esm.pretrained.esm2_t36_3B_UR50D()
batch_converter=alphabet.get_batch_converter()
_, _, batch_tokens=batch_converter(batch_sample)
batch_tokens=batch_tokens.to(self.device)
if batch_tokens.shape[1]>512:batch_tokens=batch_tokens[:,:512] #TODO random crop
# batch_labels=torch.stack(batch_labels)
batch_lens = (batch_tokens != self.alphabet.padding_idx).sum(1)
results = self.esm_model(batch_tokens, repr_layers=[self.repr_layers], return_contacts=False)
token_representations = results["representations"][self.repr_layers]
del results
sequence_representations = []
for i, tokens_len in enumerate(batch_lens):
    sequence_representations.append(token_representations[i, 1: tokens_len - 1].mean(0))
del token_representations
torch.cuda.empty_cache()
sequence_representations=torch.stack(sequence_representations)
del batch_lens,batch_tokens

1498

In [20]:
len('MVPGVPGAVLTLCLWLAASSGCLAAGPGAAAARRLDESLSAGSVQRARCASRCLSLQITRISAFFQHFQNNGSLVWCQNHKQCSKCLEPCKESGDLRKHQCQSFCEPLFPKKSYECLTSCEFLKYILLVKQGDCPAPEKASGFAAACVESCEVDNECSGVKKCCSNGCGHTCQVPKTLYKGVPLKPRKELRFTELQSGQLEVKWSSKFNISIEPVIYVVQRRWNYGIHPSEDDATHWQTVAQTTDERVQLTDIRPSRWYQFRVAAVNVHGTRGFTAPSKHFRSSKDPSAPPAPANLRLANSTVNSDGSVTVTIVWDLPEEPDIPVHHYKVFWSWMVSSKSLVPTKKKRRKTTDGFQNSVILEKLQPDCDYVVELQAITYWGQTRLKSAKVSLHFTSTHATNNKEQLVKTRKGGIQTQLPFQRRRPTRPLEVGAPFYQDGQLQVKVYWKKTEDPTVNRYHVRWFPEACAHNRTTGSEASSGMTHENYIILQDLSFSCKYKVTVQPIRPKSHSKAEAVFFTTPPCSALKGKSHKPVGCLG')

538

# proteins which does not have mutation but is in ppi

# merge Label and label and discard empty

In [12]:
df.loc[df['Label'].isna(),'Label']=df['label']

In [25]:
df=df.dropna(subset='Label')
df=df.loc[:,['#AlleleID','Label','UniProt','Name','Seq']]

# deal with Seq

In [3]:
df[df['Seq'] is None]



KeyError: False

In [6]:
df['Seq']=df['Seq'].replace(['NaN', 'None', ''], float('nan'))


In [9]:
True in df['Seq'].isna()

False

In [30]:
df['Seq']=df['Seq'].apply(lambda x: x.split('*')[0] if '*' in x else x)


In [32]:
df.to_csv('2019_1_sequences_terminated.csv')

In [ ]:
df['Len']=df['Seq'].apply(lambda x: len(x))

: 